In [1]:
print('hello')

hello


In [ ]:
import datetime
import os
import time
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.parse import urlparse, parse_qs

folder_path = "path/to/mp3/folder/"

port = 8080

def play_mp3():
    global folder_path, start_times, end_times
    
    while True:
        now = datetime.datetime.now().time()
        
        if any([start_time <= now <= end_time for start_time, end_time in zip(start_times, end_times)]):
            mp3_files = [f for f in os.listdir(folder_path) if f.endswith(".mp3")]
            for mp3_file in mp3_files:
                os.system("afplay '" + folder_path + mp3_file + "'")

class MyRequestHandler(BaseHTTPRequestHandler):
    
    def do_GET(self):
        global start_times, end_times
        
        # Parse the URL and query parameters
        parsed_url = urlparse(self.path)
        query_params = parse_qs(parsed_url.query)
        
        # Handle start request
        if parsed_url.path == "/start":
            self.handle_start_request(query_params)
        
        # Handle stop request
        elif parsed_url.path == "/stop":
            self.handle_stop_request()
        
        # Serve the HTML interface
        elif parsed_url.path == "/":
            self.serve_html_interface()

    def handle_start_request(self, query_params):
        start_time_strs = query_params.get("start_time", [])
        end_time_strs = query_params.get("end_time", [])
        
        if len(start_time_strs) != len(end_time_strs):
            self.send_response(400)
            self.end_headers()
            self.wfile.write(b"Invalid start and/or end time")
        else:
            start_times = []
            end_times = []
            for i in range(len(start_time_strs)):
                start_time = datetime.datetime.strptime(start_time_strs[i], "%H:%M:%S").time()
                end_time = datetime.datetime.strptime(end_time_strs[i], "%H:%M:%S").time()
                start_times.append(start_time)
                end_times.append(end_time)
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"Player started")

    def handle_stop_request(self):
        start_times = []
        end_times = []
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b"Player stopped")

    def serve_html_interface(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(b"""
        <!DOCTYPE html>
        <html>
          <head>
            <meta charset="utf-8">
            <title>MP3 Player</title>
            <script>
              function startPlayer() {
                var start_times = document.getElementById("start_times").value;
                var end_times = document.getElementById("end_times").value;
                var start_time_arr = start_times.split(",");
                var end_time_arr = end_times.split(",");
                var url = "/start?";
                for (var i=0; i<start_time_arr.length; i++) {
                  url += "start_time=" + start_time_arr[i] + "&";
                  url += "end_time=" + end_time_arr[i] + "&";
                }
                var request = new XMLHttpRequest();
                request.open("GET", url, true);
                request.send();
              }
              
              function stopPlayer() {
                var request = new XMLHttpRequest();
                request.open("GET", "/stop", true);
                request.send();
              }
            </script>
          </head>
          <body>
            <h1>MP3 Player</h1>
            <label for="start_times">Start times (comma-separated):</label>
            <input type="text" id="start_times" name="start_times"><br>
            <label for="end_times">End times (comma-separated):</label>
            <input type="text" id="end_times" name="end_times"><br>
            <button onclick="startPlayer()">Start player</button>
            <button onclick="stopPlayer()">Stop player</button>
          </body>
        </html>
        """)
